In [67]:
'''
INCLUDE ALL NECESSARY IMPORTS AND FUNCTIONS
'''
import keras.layers as layers
import matplotlib.pyplot as plt
from keras import Model
from numpy import argmax
from keras.optimizers import SGD
from pandas import read_csv
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.utils import to_categorical
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import numpy as np
IMAGE_SIZE=(300,300)
ALPHA=0.01
NUM_EPOCHS=5

def prepareDataOld():
  alpha=0.01
  num_epochs=5
  batch_size=32
  image_size=(300,300)
  #MAKE DF WITH IMAGE PATHS AND IMAGE LABELS
  image_dir = Path("reptiles-and-amphibians-image-dataset")
  filepaths = list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png'))
  labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
  filepaths = pd.Series(filepaths, name='Filepath').astype(str)
  labels = pd.Series(labels, name='Label')
  image_df = pd.concat([filepaths, labels], axis=1)
  
  train = ImageDataGenerator(
    rescale=1/255
  )
  validation = ImageDataGenerator(
    rescale=1/255
  )
  train_dataset = train.flow_from_directory('reptiles-and-amphibians-image-dataset', target_size={300,300}, batch_size=32, class_mode='binary')
  return


def prepareData():
  # CREATE DF WITH IMAGE PATHS AND IMAGE LABELS
  image_dir = Path("reptiles-and-amphibians-image-dataset")
  filepaths = list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png'))
  labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
  filepaths = pd.Series(filepaths, name='Filepath').astype(str)
  labels = pd.Series(labels, name='Label')
  image_df = pd.concat([filepaths, labels], axis=1)
  train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=13)

  train_generator = ImageDataGenerator(
      rescale=1./255, #normalize pixel values from 0-255 to 0-1
      validation_split=0.2 #20% of training data will be used as validation
  )
  test_generator = ImageDataGenerator(
      rescale=1./255
  )
  # CREATE DATA GENERATORS FOR TRAINING, VALIDATION, AND TESTING SETS
  train = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE, #resize all images to size 300x300
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)
  val = train_generator.flow_from_dataframe(
      dataframe=train_df,
      x_col='Filepath',
      y_col='Label',
      target_size=IMAGE_SIZE,
      color_mode='rgb',
      class_mode='categorical',
      batch_size=32,
      shuffle=True,
      seed=42,
      subset='validation'
  )
  test = test_generator.flow_from_dataframe(
      dataframe=test_df,
      x_col='Filepath',
      y_col='Label',
      target_size=IMAGE_SIZE,
      color_mode='rgb',
      class_mode='categorical',
      batch_size=32,
      shuffle=False
  )
  return train, val, test

def build_and_compile_cnn():
    model = Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300,300,3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')  
    ])
    
    sgd_optimizer = SGD(learning_rate=ALPHA)
    model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_cnn(model, train_generator, validation_generator):
    history = model.fit(
        train_generator,
        epochs=NUM_EPOCHS,
        validation_data=validation_generator
    )
    
    return history

def evaluate_model(model, generator):
    loss, accuracy = model.evaluate(generator)
    return loss, accuracy

def create_confusion_matrix(model, generator):
    y_pred = model.predict(generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = generator.classes
    confusion_matrix_result = confusion_matrix(y_true, y_pred_classes)
    return confusion_matrix_result

In [68]:
train, val, test = prepareData()
model = build_and_compile_cnn()
history = train_and_evaluate_cnn(model, train, val)
'''
train_loss, train_accuracy = evaluate_model(model, train)
validation_loss, validation_accuracy = evaluate_model(model, val)
test_loss, test_accuracy = evaluate_model(model, test)
print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Accuracy: {validation_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print("Confusion Matrix:")
confusion_matrix_result = create_confusion_matrix(model, test)
print("Confusion Matrix:")
print(confusion_matrix_result)
'''


Found 3868 validated image filenames belonging to 10 classes.
Found 967 validated image filenames belonging to 10 classes.
Found 1209 validated image filenames belonging to 10 classes.


c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  kernel_regularizer=None,


Epoch 1/5


c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  def workers(self):


 89/121 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.2941 - loss: 2.1233